# Customer Sentiment Naive Bayes Classifier

Classifier to predict whether a customer review is positive, negative, or neutral.

#### Requirements:

* pandas
* Numpy
* nltk
* re

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### 

### Quick view of the raw data:

In [3]:
reviews = pd.read_csv('iphone.csv')

In [4]:
reviews[:5]

,productAsin,country,date,isVerified,ratingScore,reviewTitle,reviewDescription,reviewUrl,reviewedIn,variant,variantAsin
0,B09G9BL5CP,India,11-08-2024,True,4,No charger,"Every thing is good about iPhones, there's not...",https://www.amazon.in/gp/customer-reviews/R345...,Reviewed in India on 11 August 2024,Colour: MidnightSize: 256 GB,B09G9BQS98
1,B09G9BL5CP,India,16-08-2024,True,5,iPhone 13 256GB,"It look so fabulous, I am android user switche...",https://www.amazon.in/gp/customer-reviews/R2HJ...,Reviewed in India on 16 August 2024,Colour: MidnightSize: 256 GB,B09G9BQS98
2,B09G9BL5CP,India,14-05-2024,True,4,Flip camera option nill,I tried to flip camera while recording but no ...,https://www.amazon.in/gp/customer-reviews/R3Y7...,Reviewed in India on 14 May 2024,Colour: MidnightSize: 256 GB,B09G9BQS98
3,B09G9BL5CP,India,24-06-2024,True,5,Product,100% genuine,https://www.amazon.in/gp/customer-reviews/R1P9...,Reviewed in India on 24 June 2024,Colour: MidnightSize: 256 GB,B09G9BQS98
4,B09G9BL5CP,India,18-05-2024,True,5,Good product,Happy to get the iPhone 13 in Amazon offer,https://www.amazon.in/gp/customer-reviews/R1XI...,Reviewed in India on 18 May 2024,Colour: MidnightSize: 256 GB,B09G9BQS98


#### 

### Clean and prepare the data:

In [8]:
column_name_mapping = {'productAsin':'Product_Number','country':'Country','date':'Date',
                       'isVerified':'Verified','ratingScore':'Rating_Score','reviewTitle':'Review_Title',
                       'reviewDescription':'Review_Description','reviewUrl':'Review_Url',
                       'reviewedIn':'Reviewer_Location','variant':'Product_Type',
                       'variantAsin':'Product_Type_Number'}

reviews.rename(columns=column_name_mapping, inplace=True)

In [9]:
# Removes emoji characters

emojis = (r'[\U0001F600-\U0001F64F'  # Emoticons
        r'\U0001F300-\U0001F5FF'  # Miscellaneous Symbols and Pictographs
        r'\U0001F680-\U0001F6FF'  # Transport and Map Symbols
        r'\U0001F700-\U0001F77F'  # Alchemical Symbols
        r'\U0001F780-\U0001F7FF'  # Geometric Shapes Extended
        r'\U0001F800-\U0001F8FF'  # Supplemental Arrows-C
        r'\U0001F900-\U0001F9FF'  # Supplemental Symbols and Pictographs
        r'\U0001FA00-\U0001FA6F'  # Chess Symbols
        r'\U0001FA70-\U0001FAFF'  # Symbols and Pictographs Extended-A
        r'\U00002764\ufe0f'  # Red Heart emoji specifically
        r']+')

reviews['Review_Description'] = reviews['Review_Description'].apply(
    lambda x: re.sub(emojis, '', str(x)) if isinstance(x, str) else x)

In [10]:
reviews['Review_Description']

0       Every thing is good about iPhones, there's not...
1       It look so fabulous, I am android user switche...
2       I tried to flip camera while recording but no ...
3                                            100% genuine
4              Happy to get the iPhone 13 in Amazon offer
                              ...                        
3057    Useless phon never buy this heat n useless cam...
3058    iam not happy with this product why because ch...
3059                                           Good phone
3060    While charging mobile it's getting so hot even...
3061    Battery power is be very bad need to chat on d...
Name: Review_Description, Length: 3062, dtype: object

In [11]:
reviews['Rating_Category'] = np.where(
    reviews['Rating_Score'] >= 4, 'High', np.where(reviews['Rating_Score'] <=2, 'Low', 'Neutral'))

In [12]:
rating_mapping = {'High':1,'Low':2,'Neutral':3}

In [13]:
reviews['Numerical_Rating_Category'] = reviews['Rating_Category'].map(rating_mapping)

In [14]:
reviews.head()

,Product_Number,Country,Date,Verified,Rating_Score,Review_Title,Review_Description,Review_Url,Reviewer_Location,Product_Type,Product_Type_Number,Rating_Category,Numerical_Rating_Category
0,B09G9BL5CP,India,11-08-2024,True,4,No charger,"Every thing is good about iPhones, there's not...",https://www.amazon.in/gp/customer-reviews/R345...,Reviewed in India on 11 August 2024,Colour: MidnightSize: 256 GB,B09G9BQS98,High,1
1,B09G9BL5CP,India,16-08-2024,True,5,iPhone 13 256GB,"It look so fabulous, I am android user switche...",https://www.amazon.in/gp/customer-reviews/R2HJ...,Reviewed in India on 16 August 2024,Colour: MidnightSize: 256 GB,B09G9BQS98,High,1
2,B09G9BL5CP,India,14-05-2024,True,4,Flip camera option nill,I tried to flip camera while recording but no ...,https://www.amazon.in/gp/customer-reviews/R3Y7...,Reviewed in India on 14 May 2024,Colour: MidnightSize: 256 GB,B09G9BQS98,High,1
3,B09G9BL5CP,India,24-06-2024,True,5,Product,100% genuine,https://www.amazon.in/gp/customer-reviews/R1P9...,Reviewed in India on 24 June 2024,Colour: MidnightSize: 256 GB,B09G9BQS98,High,1
4,B09G9BL5CP,India,18-05-2024,True,5,Good product,Happy to get the iPhone 13 in Amazon offer,https://www.amazon.in/gp/customer-reviews/R1XI...,Reviewed in India on 18 May 2024,Colour: MidnightSize: 256 GB,B09G9BQS98,High,1


In [15]:
# Splits or tokens sentence to individual words
def process_reviews(Review_Description):
    if isinstance(Review_Description,str):
        text = Review_Description.lower()
        pattern = r'\d+%|\w+|[^\w\s]'
        word_tokens = re.findall(pattern, text)
        return word_tokens
    return None

In [16]:
reviews['Review_Words'] = reviews['Review_Description'].apply(process_reviews)

In [28]:
reviews = reviews.dropna(subset=['Review_Description'])

#### 

### Exploration of processed data:

In [29]:
reviews.head()

,Product_Number,Country,Date,Verified,Rating_Score,Review_Title,Review_Description,Review_Url,Reviewer_Location,Product_Type,Product_Type_Number,Rating_Category,Numerical_Rating_Category,Review_Words
0,B09G9BL5CP,India,11-08-2024,True,4,No charger,"Every thing is good about iPhones, there's not...",https://www.amazon.in/gp/customer-reviews/R345...,Reviewed in India on 11 August 2024,Colour: MidnightSize: 256 GB,B09G9BQS98,High,1,"[every, thing, is, good, about, iphones, ,, th..."
1,B09G9BL5CP,India,16-08-2024,True,5,iPhone 13 256GB,"It look so fabulous, I am android user switche...",https://www.amazon.in/gp/customer-reviews/R2HJ...,Reviewed in India on 16 August 2024,Colour: MidnightSize: 256 GB,B09G9BQS98,High,1,"[it, look, so, fabulous, ,, i, am, android, us..."
2,B09G9BL5CP,India,14-05-2024,True,4,Flip camera option nill,I tried to flip camera while recording but no ...,https://www.amazon.in/gp/customer-reviews/R3Y7...,Reviewed in India on 14 May 2024,Colour: MidnightSize: 256 GB,B09G9BQS98,High,1,"[i, tried, to, flip, camera, while, recording,..."
3,B09G9BL5CP,India,24-06-2024,True,5,Product,100% genuine,https://www.amazon.in/gp/customer-reviews/R1P9...,Reviewed in India on 24 June 2024,Colour: MidnightSize: 256 GB,B09G9BQS98,High,1,"[100%, genuine]"
4,B09G9BL5CP,India,18-05-2024,True,5,Good product,Happy to get the iPhone 13 in Amazon offer,https://www.amazon.in/gp/customer-reviews/R1XI...,Reviewed in India on 18 May 2024,Colour: MidnightSize: 256 GB,B09G9BQS98,High,1,"[happy, to, get, the, iphone, 13, in, amazon, ..."


In [30]:
total_reviews = len(reviews)
num_positive = len(reviews[reviews['Numerical_Rating_Category']==1])
num_negative = len(reviews[reviews['Numerical_Rating_Category']==2])
num_neutral = len(reviews[reviews['Numerical_Rating_Category']==3])

print("Total Number of reviews:",total_reviews)
print("Number of positive reviews:", num_positive)
print("Number of negative reviews:", num_negative)
print("Number of neutral reviews:", num_neutral)
print()

print("Probability of positive review:", num_positive/total_reviews)
print("Probability negative review:", num_negative/total_reviews)
print("Probability of neutral review:", num_neutral/total_reviews)

Total Number of reviews: 2976
Number of positive reviews: 1994
Number of negative reviews: 747
Number of neutral reviews: 235

Probability of positive review: 0.6700268817204301
Probability negative review: 0.25100806451612906
Probability of neutral review: 0.07896505376344086


#### 

### Training the model:

In [31]:
smoothing_factor = 1

model = {}

# Train the model
for index, review in reviews.iterrows():
    if isinstance(review['Review_Words'], list):
        for word in review['Review_Words']:
            if word not in model:
                model[word] = {'Positive': smoothing_factor, 'Negative': smoothing_factor, 'Neutral':smoothing_factor}

            if word in model:
                if review['Numerical_Rating_Category'] == 1:
                    model[word]['Positive'] += 1
                elif review['Numerical_Rating_Category'] ==2 :
                    model[word]['Negative'] += 1
                else:
                    model[word]['Neutral'] += 1

In [32]:
model['amazing']

{'Positive': 136, 'Negative': 4, 'Neutral': 3}

In [33]:
for word, counts in model.items():
    total_count = counts['Positive'] + counts['Negative'] + counts['Neutral']
    model[word]['Positive'] /= total_count  # Normalize positive class probability
    model[word]['Negative'] /= total_count  # Normalize negative class probability
    model[word]['Neutral'] /= total_count   # Normalize neutral class probability

In [34]:
# Class weighting based on the class frequencies
positive_weight = num_positive / total_reviews
negative_weight = num_negative / total_reviews
neutral_weight = num_neutral / total_reviews

#### 

### Prediction:

In [35]:
def predict_customer_sentiment(review_text):
    review_words = process_reviews(review_text)
    if not review_words:
        return "Neutral"
    
    positive_prob = np.log(positive_weight)
    negative_prob = np.log(negative_weight)
    neutral_prob = np.log(neutral_weight)
    
    for word in review_words:
        if word in model:
            positive_prob += np.log(model[word]['Positive'])
            negative_prob += np.log(model[word]['Negative'])
            neutral_prob += np.log(model[word]['Neutral'])
            
    if positive_prob > negative_prob and positive_prob > neutral_prob:
        return 'Positive'
    elif negative_prob > positive_prob and negative_prob > neutral_prob:
        return 'Negative'
    else:
        return 'Neutral'

#### 

### Testing:

In [36]:
testreview1 = "I absolutely love my new iPhone!"
print(predict_customer_sentiment(testreview1))

Positive


In [37]:
testreview2 = "I am extremely disappointed with the iPhone. The battery life is bad."
print(predict_customer_sentiment(testreview2))

Positive


In [38]:
testreview3 = "The phone is ok."
predict_customer_sentiment(testreview3)

'Positive'

### 